In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import time
import serial
import csv
import numpy as np
import matplotlib.pylab as plt
from matplotlib import colors
from IPython import display
import statistics

def sendcmd(command, resp=True):
    ser = serial.Serial(
        port='/dev/ttyUSB0',
        baudrate=57600,
        parity=serial.PARITY_ODD,
        stopbits=serial.STOPBITS_ONE,
        bytesize=serial.SEVENBITS
    )
    if ser.isOpen() == 0:
        ser.open()
    comando = command + '\r\n'
    #print('comando: ' + comando)
    ser.write(comando.encode())  # Encode the command as bytes before writing
    out = b''  # Initialize out as bytes
    time.sleep(0.1)  # Wait before reading output
    while ser.inWaiting() > 0:
        out += ser.read(1)
    ser.close()
    return out.decode()  # Decode bytes into a string before returning

def printVals(timeStamp, temp):
    if isinstance(slope, float) == False:
        print(end='\x1b[2K') #clear last line
        print('time: ', round(timeStamp[-1],2), 
              ' temp: ', round(temp[-1],3),
              ' output: ', round(newOut,2),
              end='\r')
        row = [timeStamp[-1], temp[-1], newOut]
    else:
        print(end='\x1b[2K') #clear last line
        print('time: ', round(timeStamp[-1],2),
              ' temp: ', round(temp[-1],3),
              ' output: ', round(newOut,2),
              ' slope: ', round(slope,6),
              ' variance: ', round(varTemp,6),
              end='\r')
        row = [timeStamp[-1], temp[-1], newOut, slope, varTemp]
    return row

In [3]:
run = '13'
filenameData = "tuningDataNB" + str(run) + ".csv"
fileheadersData = ['time', 'temp', 'output', 'slope', 'variance']
with open(filenameData, 'w', newline='') as csvfile:  
    writer = csv.writer(csvfile) 
    writer.writerow(fileheadersData) 
timeStamp = []; temp = []
#stdTempLim = 0.01

#outVals = np.arange(55.0,61.0,1)
#out = outVals.tolist()
#out.append(0.0) #last output is 0 to cool down
out = np.arange(0, 40, 5).tolist()
out = out + np.arange(40, 50, 2).tolist()
out = out + np.arange(50, 55, 1).tolist()
out = out + np.arange(50, 60.5, 0.5).tolist()

print(out)
t_start = time.time()
#offTime = 60 * 5 #time heater is off before starting
checkRange = 60*10 #time to measure slope/std dev of temp over
varTempLim = 0.005 #Variation allowed for steady temperature (K)
slopeLim = varTempLim/checkRange #Net slope allowed for steady temperature
timeLimit = 60*60*2 #maximum time to spend on one output
timeInterval = 1

for newOut in out:
    #set next output
    sendcmd('MOUT 1,' + str(newOut))
    newOutIndex = len(timeStamp)
    print(end='\x1b[2K')
    print("output ", newOut, " at index ", newOutIndex, end='\n') 
    #keep output until steady temperature is reached
    slope = None
    steady = False
    while steady == False:
        #update time and temp reading
        timeStamp.append(time.time() - t_start)
        temp.append(float(sendcmd('KRDG? A')))
        #if (newOut == 0.0) and (timeStamp[-1] >= timeOff):
        #    print(end='\x1b[2K')
        #    print('Turning heater on', end='\n')
        #    break
        #if the checkRange time has passed since new output
        if timeStamp[-1] - timeStamp[newOutIndex] >= checkRange:
            #find index of data from checkRange seconds ago
            timeDiffArr = np.absolute(np.array(timeStamp) 
                                      - timeStamp[-1] + checkRange)
            index = timeDiffArr.argmin()
            #calculate slope of temperature over the checkRange
            trend = np.polyfit(timeStamp[index:], temp[index:], 1)
            slope = trend[0]
            varTemp = statistics.variance(temp[index:])
            #stdTemp = np.std(temp[index:])
            #If the slope is level and there is little variation
            if ((abs(slope) <= slopeLim) and (varTemp <= varTempLim)):
                #Then record end avg temp and move on to next output
                steady = True
                print(end='\x1b[2K')
                print("Steady temp reached.", 
                      '                                         ', 
                      end='\n')
            #If too much time has passed and steady temp has not been reached
            elif timeStamp[-1] - timeStamp[newOutIndex] >= timeLimit:
                #Then record end avg temp and move on to next output
                print(end='\x1b[2K')
                print("Steady temp not reached. Time limit exceeded.", 
                      '                                 ',
                      end='\n')
                break
        #print current data and store it in a list
        dataRow = printVals(timeStamp, temp)
        #write time, temp, slope data to csv
        with open(filenameData, 'a', newline='') as csvfile:  
            writer = csv.writer(csvfile) 
            writer.writerow(dataRow)
        time.sleep(timeInterval)
print("done")

[0, 5, 10, 15, 20, 25, 30, 35, 40, 42, 44, 46, 48, 50, 51, 52, 53, 54, 50.0, 50.5, 51.0, 51.5, 52.0, 52.5, 53.0, 53.5, 54.0, 54.5, 55.0, 55.5, 56.0, 56.5, 57.0, 57.5, 58.0, 58.5, 59.0, 59.5, 60.0]


SerialException: could not open port '/dev/ttyUSB0': FileNotFoundError(2, 'The system cannot find the path specified.', None, 3)